In [1]:
import os
import torch
import cv2
import torchvision.transforms as transforms
from torch.utils.data import Dataset , DataLoader, random_split
from dotenv import load_dotenv
import numpy as np
import torch.nn as nn
import torch.optim as optim
from imgDataset import imgDataset
from EAST import EAST
from train import train_model

In [2]:
load_dotenv()
directory_train = os.getenv('Directory_train')
directory_train_textAndCoords = os.getenv('directory_train_textAndCoords')
directory_test = os.getenv('Directory_test')
directory_test_textAndCoords = os.getenv('directory_test_textAndCoords')


In [3]:
# Define path_imgs and path_labels
path_imgs = os.listdir(directory_train)
path_labels = os.listdir(directory_train_textAndCoords)

# Generate paths for training images and labels
train_img_paths = [os.path.join(directory_train, f) for f in path_imgs]
train_label_paths = [os.path.join(directory_train_textAndCoords, f) for f in path_labels]

#print(train_img_paths)
#print(train_label_paths)

In [4]:
train_dataset = [train_img_paths, train_label_paths]
train_dataset = imgDataset(train_img_paths, train_label_paths)
img, labels, coords = train_dataset[0]

print(f"Image shape: {img.shape}") # Should be [1, 360, 360] since it's grayscale
print(f"First Label: {labels}")
print(f"First Coords: {coords}")

['377', '117', '463', '117', '465', '130', '378', '130']
['493', '115', '519', '115', '519', '131', '493', '131']
['374', '155', '409', '155', '409', '170', '374', '170']
['492', '151', '551', '151', '551', '170', '492', '170']
['376', '198', '422', '198', '422', '212', '376', '212']
['494', '190', '539', '189', '539', '205', '494', '206']
['374', '1', '494', '0', '492', '85', '372', '86']
Image shape: torch.Size([1, 640, 360])
First Label: [['G', 'e', 'n', 'a', 'x', 'i', 's', ' ', 'T', 'h', 'e', 'a', 't', 'r', 'e'], ['[', '0', '6', ']'], ['#', '#', '#'], ['6', '2', '-', '0', '3'], ['C', 'a', 'r', 'p', 'a', 'r', 'k'], ['#', '#', '#'], ['#', '#', '#']]
First Coords: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  

In [ ]:
#spltining the dataset into training and validation sets
val_split = 0.2
training_size = int((1 - val_split) * len(train_dataset))
val_size = len(train_dataset) - training_size
training_dataset, val_dataset = random_split(train_dataset, [training_size, val_size])
loader_train = DataLoader(training_dataset, batch_size=32, shuffle=True)
loader_val = DataLoader(val_dataset, batch_size=32, shuffle=True)

#print(f"loader_train: {loader_train}")
#print(f"loader_val: {loader_val}")

In [ ]:
model = EAST(color_channel=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
#loasding model if it exists
if os.getenv('Load_model') == 'True': 
    model.load_state_dict(torch.load(os.getenv('model_path')))
    print("Model loaded successfully")
else:
    print("Model not load, starting from scratch")

In [ ]:
#defualt model otipions
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
cycles = 12
train_model(model = model, loader_train = loader_train, loader_val = loader_val, criterion = criterion, optimizer = optimizer, cycles = cycles)

In [ ]:
model_save_path = os.getenv('Model_save_path', 'east_model.pth')
torch.save(model.state_dict(), model_save_path)